In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
subset_test_path = './new_train/train_subset'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def train_agents_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:],scene['v_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:],scene['v_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def train_all_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:],scene['v_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])     
    out = numpy.concatenate([numpy.dstack([scene['p_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:],scene['v_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]
    
def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:],scene['v_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
train_agent_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_agents_collate, num_workers=0)

train_all_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_all_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 4,
                 embedding_size = 16,
                 hidden_size = 1792):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=16, hidden_size=1792, output_size=4):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = self.linear1(x)
        embedded = F.relu(x)
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(iterator):
        
        inp, out = sample_batch
#         print(inp.shape)
        # preprocessing more ????
#         inp = inp[:,0,:,:]
#         out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #output whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :]
        
        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden
#         print("DECODER INPUT", decoder_input.shape)
#         print("DECODER HIDDEN", decoder_hidden[0].shape)
        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :4], target[:, di, :4]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         file1 = open("loss_steps.txt", "a")  # append mode
#         file1.write(str(loss.item()) + ",")
#         file1.close()
        
#       output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        if counter % (0.1*len(train_loader)) == 0:
            print("10% passed")
        iterator.set_postfix(loss=(loss.item()))
    

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
#         _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :4]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 4)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [6]:
import pandas as pd
import numpy as np

device = "cuda"
encoder = EncoderRNN(input_size=4)
decoder = DecoderRNN(output_size=4)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 10

gamma=0.5

for epoch in range(1, num_epoch + 1):
    for param_group in encoder_optimizer.param_groups:
        if epoch % 1 == 0:
            param_group['lr'] *= gamma
    for param_group in decoder_optimizer.param_groups:
        if epoch % 1 == 0:
            param_group['lr'] *= gamma
    train(encoder, decoder, device, train_all_loader, encoder_optimizer, decoder_optimizer, epoch)
#         train(encoder, decoder, device, train_agent_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)
    
    output = infer_absolute(test_loader, encoder, decoder)
    df = pd.DataFrame.from_dict(output, orient='index')
    for i in range(30):
        df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)[[0, 1]]
    dropped_cols = list(np.arange(30))
    df2 = df.drop(dropped_cols, axis=1)
    df2.index.name = 'ID'
    df2.to_csv("outputs" + str(epoch) + "epb4_v_hidden1792_all_gamma1.csv", index=True)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
9129,"[317.60418701171875, 1525.259521484375, 3.4585...","[317.8072509765625, 1525.2398681640625, 3.0443...","[318.2269592285156, 1525.32861328125, 3.057605...","[318.086669921875, 1526.1390380859375, 2.96607...","[318.5282897949219, 1525.9857177734375, 3.0541...","[318.626708984375, 1526.3692626953125, 3.00288...","[319.036865234375, 1526.4613037109375, 3.06671...","[319.3877868652344, 1526.77294921875, 3.257619...","[319.5994873046875, 1526.844482421875, 3.33451...","[319.9951171875, 1527.0263671875, 3.4094336032...",...,"[324.6156311035156, 1528.5826416015625, 4.1640...","[325.0142517089844, 1528.5537109375, 4.0629000...","[325.56591796875, 1528.8214111328125, 4.034102...","[325.74957275390625, 1528.8446044921875, 4.180...","[326.1426086425781, 1528.954345703125, 4.13698...","[326.5916748046875, 1529.096435546875, 4.13042...","[326.9604187011719, 1529.365234375, 4.15209102...","[327.3158874511719, 1529.599365234375, 4.18753...","[327.7618103027344, 1529.752197265625, 4.24817...","[328.1631164550781, 1529.8988037109375, 4.3073..."
7849,"[1799.812255859375, 408.2670593261719, 7.33698...","[1800.6907958984375, 409.1272888183594, 8.4957...","[1801.5255126953125, 409.65081787109375, 8.603...","[1802.162109375, 410.60211181640625, 8.5611438...","[1803.064208984375, 411.1955871582031, 8.55706...","[1804.0872802734375, 412.1924743652344, 8.4437...","[1804.7041015625, 412.6981201171875, 8.3520641...","[1805.533935546875, 413.5502014160156, 8.39219...","[1806.4417724609375, 414.24365234375, 8.200574...","[1807.3701171875, 414.9962158203125, 8.1241092...",...,"[1816.2705078125, 422.5098876953125, 8.2743320...","[1816.99560546875, 423.28424072265625, 8.19789...","[1817.743896484375, 424.052978515625, 8.155121...","[1818.530029296875, 424.5613098144531, 8.08820...","[1819.2723388671875, 425.27410888671875, 8.010...","[1819.9405517578125, 426.0450439453125, 8.0007...","[1820.917724609375, 426.8339538574219, 8.00844...","[1821.42626953125, 427.43695068359375, 8.05086...","[1822.095458984375, 427.6315612792969, 8.02326...","[1823.00732421875, 428.4291076660156, 7.940543..."
30287,"[1793.746826171875, 402.8463134765625, 11.6602...","[1794.6610107421875, 404.0157165527344, 11.616...","[1795.983642578125, 405.0002746582031, 11.5083...","[1797.2110595703125, 405.9278564453125, 11.665...","[1798.3560791015625, 406.8338317871094, 11.660...","[1799.571533203125, 407.86883544921875, 11.725...","[1800.85986328125, 408.87689208984375, 11.7066...","[1802.06689453125, 409.8540344238281, 11.76632...","[1803.115966796875, 410.8739318847656, 11.7328...","[1804.34423828125, 411.8505859375, 11.69383430...",...,"[1817.23876953125, 422.69891357421875, 11.6961...","[1818.4537353515625, 423.7760009765625, 11.632...","[1819.721435546875, 424.8702392578125, 11.6751...","[1820.7071533203125, 425.8349609375, 11.699050...","[1822.014404296875, 426.9884033203125, 11.6613...","[1823.191650390625, 427.83831787109375, 11.637...","[1824.2598876953125, 428.9688415527344, 11.672...","[1825.639404296875, 429.8393859863281, 11.5685...","[1826.7252197265625, 430.9634094238281, 11.428...","[1827.6063232421875, 431.68426513671875, 11.25..."
23642,"[2078.5537109375, 641.5218505859375, -11.63940...","[2077.61669921875, 640.5254516601562, -8.95072...","[2076.83447265625, 639.3902587890625, -8.75510...","[2075.7822265625, 638.9707641601562, -8.859302...","[2074.960693359375, 638.4669189453125, -8.7928...","[2073.97607421875, 637.57275390625, -8.8946628...","[2073.087158203125, 636.9010620117188, -8.7356...","[2071.893798828125, 636.0731201171875, -8.7331...","[2071.139404296875, 635.2140502929688, -8.6360...","[2070.229736328125, 634.4637451171875, -8.7951...",...,"[2059.97998046875, 626.188232421875, -8.696512...","[2059.33203125, 625.5569458007812, -8.68755531...","[2058.211669921875, 624.8334350585938, -8.6457...","[2057.4951171875, 624.13818359375, -8.70736026...","[2056.253662109375, 623.2434692382812, -8.7184...","[2055.

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)[[0, 1]]

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
9129,317.604187,1525.259521,317.807251,1525.239868,318.226959,1525.328613,318.086670,1526.139038,318.528290,1525.985718,...,326.591675,1529.096436,326.960419,1529.365234,327.315887,1529.599365,327.761810,1529.752197,328.163116,1529.898804
7849,1799.812256,408.267059,1800.690796,409.127289,1801.525513,409.650818,1802.162109,410.602112,1803.064209,411.195587,...,1819.940552,426.045044,1820.917725,426.833954,1821.426270,427.436951,1822.095459,427.631561,1823.007324,428.429108
30287,1793.746826,402.846313,1794.661011,404.015717,1795.983643,405.000275,1797.211060,405.927856,1798.356079,406.833832,...,1823.191650,427.838318,1824.259888,428.968842,1825.639404,429.839386,1826.725220,430.963409,1827.606323,431.684265
23642,2078.553711,641.521851,2077.616699,640.525452,2076.834473,639.390259,2075.782227,638.970764,2074.960693,638.466919,...,2055.475342,622.523743,2054.490967,621.621216,2053.652588,620.888123,2052.437012,620.003845,2051.896973,619.569580
29837,1775.022095,391.041809,1775.892212,391.973602,1776.837769,392.719147,1777.732178,393.456757,1778.609741,394.091705,...,1797.414185,410.041138,1798.459229,410.909515,1799.311523,411.546265,1800.249756,412.382751,1800.965698,413.099731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37429,415.603058,1370.529907,415.660858,1371.921265,415.675537,1373.199097,415.616699,1374.616455,415.508789,1375.818604,...,414.797394,1402.747070,414.627655,1403.895996,414.592377,1405.195801,414.607697,1406.601318,414.498688,1407.274170
18899,2015.990234,591.995789,2016.891968,592.906311,2017.729858,593.642273,2018.610229,594.524902,2019.607788,595.063293,...,2039.236938,611.496155,2040.308350,612.344482,2041.323608,613.090454,2042.196777,613.813599,2042.782104,614.463257
11528,1881.291382,487.580505,1880.405640,486.853119,1879.946289,485.854797,1878.644653,485.576324,1877.673950,484.804749,...,1859.089844,468.576935,1857.962646,467.743073,1857.485962,467.120483,1857.131348,466.546875,1856.116821,465.929779


In [12]:
df2.to_csv("outputs2epb4_v_hidden1792_all_gamma01.csv", index=True)